# Week 8. Reinforcement Learning assignment
# : Playing Atari with Deep Reinforcement Learning 논문 리뷰

### 13기 이유민

---

# [ Contents ]

## Preview

##  Playing Atari with Deep Reinforcement Learning Review
### 0. Abstract
### 1. Introduction
    - Experience Replay
    - Q-Learning & Deep-Learning
### 2. Background
    - Markov Decision Process & Discounted Future Reward
    - Bellman Equation
    - E-greedy Algorithm
    - Loss Function
### 3. Related Work
### 4. Deep Reinforcement Learning
    - 4.1. Preprocessing and Model Architecture
        - 1. Preprocessing
        - 2. Model Architecture
        - 3. DNN Architecture
### 5. Experiment
    - 5.1 Training and Stability
    - 5.2 Visualizing the Value Function
    - 5.3 Main Evaluation
### 7. Conclusion
    - DQN algorithm code


## 8. Reference

## 참고한 논문

---

# Preview
### <논문을 통해 얻고자 하는 것> 
- DQN이 무엇이고 
    - convolutional networks trained with our approach as Deep Q-Networks (DQN)
- 어떤 인자를 받아서
    - raw pixel
- 어떤 네트워크를 통과해서
    - Convolutional Neural Network (CNN)
- 어떤 결과를 내는지
    - value function (가치함수)


또 Atari game에 대해 전혀 몰랐기 때문에 관련 영상 몇 가지를 찾아 본 후, 그 중 하나를 첨부했다.

In [1]:
from IPython.display import YouTubeVideo
YouTubeVideo('https://www.youtube.com/watch?v=V1eYniJ0Rnk')

영상 출처 - https://youtu.be/V1eYniJ0Rnk (실행되지 않으면 참고해 주세요!)

---

# 0. Abstract

<img src="https://adeshpande3.github.io/assets/IRL10.png">

이미지 출처 - https://adeshpande3.github.io/assets/IRL10.png

Deep Learning과 Reinforcement Learning을 합쳐 Atari라는 게임 종류 중 2600개가 넘는 게임에 같은 모델과 학습 알고리즘을 적용시켜 의미있는 결과를 이끌어낸 논문이다.

모델은 스크린의 픽셀값들을 입력으로 받고, 각 action에 대해 점수를 부여하고, 어떤 행동에 대한 결과값을 함수를 통해 받는 방식으로 게임을 학습한다.

여기에서 CNN 모델을 사용했고, 변형된 Q-learning(Q-learning에서 input은 raw pixels, output은 미래의 보상을 예측하는 가치 함수이다.)으로 학습시켰다.

논문 내용을 그대로 직역하면 'High-Dimensional Sensory Input으로부터 Reinforcement Learning를 이용해서 Control Policy를 성공적으로 학습하는 Deep Learning Model을 소개한다.' 정도로 이해할 수 있겠다. 

__요약: DQN은 Atari 게임을 할 때, raw pixel을 input값으로 받은 후, CNN을 function approximator로 이용하여, 미래 reward를 추정하는 데 쓰는 가치함수를 output으로 낸다.__


---

# 1. Introduction

__이 논문에서는__ 

1. CNN을 통해 복잡한 RL 환경에서 원시 비디오로부터 성공적으로 Control Policy를 학습할 수 있음을 증명

2. Variant Q-Learning을 사용하며, weight를 update하기 위해 SGD(Stochastic Gradient Descent) 사용

3. 상관관계가 있는 Data와 Non-stationary distributions의 문제를 해결하기 위해 Experience Replay 사용

4. 하나의 Neural Network를 사용하였고, 게임에 대한 특정 정보나 데이터를 제공하지 않고
시각 데이터와 Reward 그리고 터미널로부터 오는 신호, 행동들로 만 학습을 진행

5. 동일한 Network Architecture와 Hyperparameter를 사용하여 다양한 게임의 학습 진행

딥러닝이 발전하면서 Vision, Speech와 같은 고차원의 데이터들을 입력으로 사용하여 CNN, Multi-Layer Perceptrons, restricted Boltzmann machines, recurrent neural networks 등을 통해 지도 및 비지도학습에 사용할 수 있게 되었다. 

하지만 RL에 DL을 적용할 때 발생하는 문제점을 해결해야 하는데, 이는 크게 두 가지로 정리할 수 있다. (한 가지가 더 있긴 하지만 1,2에 비해서는 작은 문제이기 때문에 언급하지 않고 넘기겠습니다!)

1. 보통 DL applications는 hand-labelled training data(이미 라벨링된 데이터) 를 사용한다. 하지만 RL은  reward를 통해서만 학습하고, reward 또한 부족하며 고르지 않고 delay되어 주어진다.

2. DL에서의 data sample은 Independent and identically distributed 한 분포를 따른다고 가정한다. 그에 반해 RL에서는 현재 state 상태에 따라서 가능한 다음 state가 결정하기 때문에 state 사이의 상관관계와 data 사이의 상관관계가 크다. 

이 논문은 방금 언급한 두 문제점을 해결해서 RL에 DL을 적용한다.

여기서 사용된 방법이 'Experience Replay'다.

### * Experience Replay ?

DQN으로 Q함수를 근사시키면 local minimum에 빠지거나, 혹은발산하는 문제가 있을 수 있다.

이러한 문제를 보완하는 방법이 experience replay 기법으로, 개념만 보면 mini-batch와 비슷하다.

먼저 게임 과정의 전이 <s, a, r, s'>들을 모두 replay 메모리에 저장한다. 그리고 학습단계에서 최신 전이들을 하나씩 사용하는 대신, replay 메모리에서 랜덤으로 여러개를 선택해서 학습하는 방식이다.

### * Q-Learning & Deep-Learning 이란?

<img src='https://poqw.github.io/assets/images/dqn_2.png'>
이미지 출처 - https://poqw.github.io/DQN/

그림은 인공지능이 특정 state에서 어떤 action을 취하는 게 좋을지 policy를 세우고, 그 policy에 따라서 action을 한다는 것을 표현하고 있다. 

이 policy를 세우는 과정에서 인공지능은 가장 reward가 높은 action을 선택하게 되는데, 이 때 사용되는 게 π이다. π는 2.Background에서 좀 더 자세히 다루겠다.

인공지능이 reward를 정하는 방법은 __"자기 자신에게 잘 하고 있는지 물어보는"__ 것이다. 

예를 하나 들어보자.

엣지 오르 투모로우 라는 영화에서 주인공은 전쟁터에서 죽을 때마다 지금까지 있었던 모든 기억을 가지고 다시 살아나는 무한 루프에 빠진다. 이 기억을 토대로 다시 살아나는 과정이 반복될수록 오래 살아남게 된다.
여기서 기억은 state, action은 살아남기 위해 하는 행동, reward는 살아남은 시간으로 이해할 수 있다. 

Q-Learning에서 s, a를 모두 가지고 있는 Q-tanle은 비효율적이기 때문에 NN을 사용하게 된다. s, a가 필요했던 Q러닝과는 다르게 NN에서는 이 둘을 집어넣으면 r값이 나오게 하자는 아이디어이다.

(이미 너무 자주 봤지만!) DNN의 형태는 다음과 같다. 
<img src='https://poqw.github.io/assets/images/dqn_6.png'>
이미지 출처 - https://poqw.github.io/DQN/

<img src='https://www.intel.ai/wp-content/uploads/sites/69/deep-q-network-example.png'>
이미지 출처 - https://www.intel.ai/demystifying-deep-reinforcement-learning/#_ftn1


NN에 대해서는 여러 번 다뤄봤으므로 여기서는 __s와 a로 r을 얻을 수 있다__ 는 정도만 기억해두고 2.Background로 넘어가자!

---

# 2. Background

이 논문은 MDP(Marcov Decision Process)를 중심으로, 그 중에서도 특히 Q-learning algorithm을 NN(Neural Network)를 통해 해결한다.

Agent가 환경(E, Atari Emultator)와 상호작용하는 task가 있다고 할 때, 

1. Agent는 매 time-step마다 할 수 있는 Action($a_{t}$) set에서 하나를 선택
2. Action이 전해지고 나면 Emulator는 state를 변경 & game score 수정
  - 여기서 Agent는 현재 게임이 어떤 상태인 지는 알 수 없음.
  - Agent는 이미지와 Reward($r_{t}$)만 전달받음
  - 이미지 : raw poxel의 벡터로 이루어져 있음(현재 화면)
  - Reward : game score의 변화

3. 정리하자면 Agent가 매 time-step마다 action을 선택하면 Emulator를 통해 state를 수정하고 reward($r_{t}$)가 return 되는 것!

---

### * Markov Decision Process & Discounted Future Reward

__우리가 하고자 하는 것? MDP로 정의된 문제에 대해 매 state마다 전체 reward를 최대화하는 action을 결정하는 것!__

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile3.uf.tistory.com%2Fimage%2F99B1D7485ACF62550E6AE5'>
이미지 출처 - https://untitledtblog.tistory.com/139

위 그림은 MDP가 어떻게 동작하고 있는지를 구조화해서 나타내고 있다.

agent는 $S_{t}$ 에 해당하는 state에서 $A_{t}$ 에 해당하는 action을 수행한다. 그러면 게임 법칙, 물리 엔진 등에 해당하는 environment는 다음 state에 해당하는 $S_{t+1}$ 과 그에 상응하는 reward $R_{t+1}$을 agent에게 반환한다.


---

1. $R_{t}$
 : 시간 t에서 discounted factor가 정의된 reward
 - $R_{t}=∑r^{t^{'}-t}r_{t}$
   
 - T는 game이 종료되는 time
 - 현재의 시점에서 종료 시점까지 reward값에 reward factor의 (t'-t) 지수승 값


2. $Q^{*}(s,a)$
  : action을 통해 얻을 수 있는 기대값의 최대치를 반환하는 최적의 action-value function
  - $Q^{*}(s,a) = max_{π}E[R_{t}|s_{t}=s,a_{t}=α,π]$

  - π: $s_{t}$에서 $a_{t}$를 매핑해주는 정책함수

---

### * Bellman Equation

참조 : https://github.com/yohan5050/MAL/wiki/%5B%EA%B0%95%ED%99%94%ED%95%99%EC%8A%B5%5D4.-Bellman-Equation-(%EB%B2%A8%EB%A7%8C-%EB%B0%A9%EC%A0%95%EC%8B%9D)

최적의 Q-Function은 Bellman Equation을 따른다.  
벨만 방정식의 특징은 크게
- 1. policy과 value를 고려한다. 
- 2. 특정 policy 에서 기대되는 미래 보상의 합을 의미하는 방정식이다. 
        여기서 미래 보상은 value를 의미한다.
- 3. 방정식은 value iteration 에서 사용된다.


이 방정식은 sequence s′의 다음 time-step에서 최적의 $Q^{*}(s,a)$ 값이 모든 모든 행동 a′ 에 알려져 있다면, 최적의 전략은 $ r+γQ^{*}(s,a)$ 의 Expected Value를 최대화한다.

수식으로 표현하면 다음과 같다. 

$Q^{*}(s,a)=E_{s′∼e}[r+γmaxa′Q^{∗}(s′,a′) ∣∣ s,a].$

많은 강화학습 알고리즘에서는 Q-Function을 추정하기 위하여 벨만 방정식을 사용한다.

이 때 쓰이는 Value Iteration 알고리즘은 매 i번째 Iteration마다 아래와 같은 프로시저를 수행한다.

$Q^{*}(s,a)=E[r+γQ_{i}(s′,a′) ∣∣ s,a]$

이러한 Value Iteration Algorithm은 MDP에서

$Q_{i}→Q^{∗}$ as $i→∞$ 이다.

그러나 action-value function은 각 sequence마다 독립적으로 측정되기 때문에, 이런 방식은 실제로 impractical(비현실적)하다. 대신 function approximator를 사용하여 action-value function을 적절히 approximate 시킨다.  
 
$Q(s,a;θ)≃Q^{∗}(s,a)$

일반적으로는 Linear Function으로 Approximate 하지만, 간혹 Non-Linaer Function으로 Approximate하는 경우도 있다. 

neural network function approximator로 weight θ 를 사용하는 것을 Q-network라고 한다. Q-Network는 each iteration마다 바뀌는 Loss function $L_{i}(θ_{i})$를 최소화시킴으로써 학습을 한다. 

$L_{i}(θ_{i}) = E_{s},a∼ρ(⋅)[(y_{i}−Q(s,a;θ_{i}))2]$,

여기서 $y_{i}=E_{s}∼E[r+γmax_{a} Q(s′,a′;θ_{i−1}) ∣∣ s,a]$

여기서 yi 는 iteration i의 target value이며 ρ(s,a) 는 behaviour distribution으로 sequence s에 대해 action a의 probability distribution이다.

$θ_{i−1}$ 은 Loss function $L_{i}(θ_{i})$를 optimize할 때 fixed 되는데, 학습이 진행되기 전에 고정되었던 supervised learning과는 대조된다. 이러한 네트워크의 Gradient는 아래와 같다. 

∇θ_{i}L_{i}(θ_{i)=$E_{s,a∼ρ(⋅);s′∼E}[(r+γmax_{a′}Q(s,a;θ_{i−1})−Q(s,a;θ_{i}))$∇$θ_{i}Q(s,a;θ_{i})]$

Deep-Q Learning은 e(입실론)과는 따로 돌아가는 model-free 알고리즘이다.

또한 Off-Policy를 사용한다. (Behavior Policy와 Learning Policy를 별도로 두는 policy)

ϵ의 확률로 random action을 선택하고, .1−ϵ의 확률로는 a=maxaQ(s,a,;θ) 인 greedy strategy를 따른다. 

여기서 일반적으로 RL의 데이터들은 correlation이 상당히 높다. 그러므로 이러한 data들간의 correlation을 없애주기 위해 일정한 확률로 random action을 선택하고, 남은 확률로는 greedy strategy를 따라 행동을 선택한다. 

그리디 알고리즘에 대해 좀 더 알아보자.

### * E-greedy Algorithm (입실론 그리디 알고리즘)
참조 : https://medium.com/analytics-vidhya/the-epsilon-greedy-algorithm-for-reinforcement-learning-5fe6f96dc870

: https://mc.ai/introduction-reinforcement-learning-with-epsilon-greedybandit-gamealgorithm/

<img src='https://miro.medium.com/max/1000/0*1gBOEmLxp9jWuH98'>
이미지 출처 - https://www.endtoend.ai/slowpapers/cer/


epsilon-greedy exploration으로 설명해보자.

Q-learning 알고리즘에서 초기 값(행렬 또는 네트워크 파라미터)는 모두 랜덤이다.

따라서 가장 높은 Q값을 가지는 action을 선택한 경우, action 또한 랜덤으로 선택한 것과 같은 효과를 가지며, agent는 exploration을 수행한다. 

반복을 통해 Q함수가 수렴하게 되면, 이러한 exploratory action은 줄어들게 된다.

이처럼 Q-learning 알고리즘 자체는 exploration 개념을 포함하고 있지만, exploration을 통해 최초 발견한 전략에서 벗어나지 못한다.

epsilon-greedy exploration은 이러한 문제를 해결하는 기법이다. 

기본적으로는 가장 높은 Q값을 가지는 action을 선택하고, 일정 확률(입실론)로 랜덤하게 action을 선택하게 한다.

학습 초기에는 랜덤으로 action을 선택하고, Q-functon이 수렴하면 exploration은 고정된 비율로만 시도할 것이다.

### * Loss Function

Loss는 다음 상태에 대한 Q의 예측값이다.

Loss는 s'과 s''를 고려해서 계산할 수 있고, Q는 현재의 예측값이며, state나 action을 고려하여 최대의 결과를 추출한다. 

여기서는 $Q = r+γ(max(Q(s′,a′)))$ 에 해당한다.

---
# 3. Related Work

Reinforcement가 적용되었던 가장 유명한 사례는 TD-gammon가 있다. 

강화학습을 통해 스스로 플레이 방법을 터득하고, Q-learning과 유사하게 Model-Free한 구조로,  TD 오류의 역 전파에 의해 학습 된 multilayer neural network을 사용하는nonlinear function approximation의 간단한 조합이다.

하지만 TD-gammon의 방식을 GO나 Chess에 적용을 할 때면 실패하였다. 

즉, 이러한 접근법은 TD-Gammon에만 최적화되어있다고 볼 수 있다.

또한 Q-Learning과 같은 Model-Free 강화학습 알고리즘을 Non-Linear Function Approximator나 Off-Policy Learning에 적용시키면 Q-network가 발산하는데  이를 수렴시키기 위해 주로 Linear Function Approximators에 초점을 두고 RL이 진행되도록 한다.

gradient temporal-difference를 이용해 발산문제가 다루어지게 되었고, 이를 이용하여 non-linear function approximator로 fixed policy를 사용하거나 제한적인 Q-Learning을 변형하여 linear function approximation를 이용해 control policy를 학습할 때 수렴함을 증명하였다. 

그러나 이것은 nonlinear control 까지 확정되지 않은 상태였다.

이와 가장 유사했던 작업으로는 Neural fitted Q-Learning(NFQ)가 있다.
 
여기서 사용한 Batch Gradient Descent는 iteration을 돌기 위해 많은 계산이 필요하다는 단점이 있었기에 논문에서는 Iteration을 돌기 위해 필요한 계산 양을 줄이기 위헤           Stochastic Gradient Descent를 사용하였다. 

결과적으로 큰 Data-Set까지 학습을 Scale-up 시킬 수 있었다. 

처음으로 deep auto encoder를 사용함으로써 NFQ는 task의 low dimensional representation을 학습할 수 있었고, 시각입력을 사용하여 real-world control task를 성공적으로 NFQ 알고리즘에 적용할 수 있었다.

그러나 이와 반대로 우리는 시각적 입력으로부터 직접적으로 강화학습을 적용시켰기 때문에 
Action-Value를 판별하는 것과 같은 특징들을 학습하였다.

많은 특징들을 lower-dimensional space에 적용시키면서 결과도 점점 좋아졌다.
 
HyperNEAT evolutionary 아키텍처는 Atari Game의 플랫폼에도 적용되었으며, 이 플랫폼은 게임마다 전략을 나타내는 NN을 발전시키는데 사용되었다. 

Emulator의 리셋 기능을 사용하여 결정을 내리는 Sequence에 대해 반복적으로 교육을 받을 경우 이러한 전략은 여러 atari 게임에서의 디자인 결함을 악용할 수 있다.

---

# 4. Deep Reinforcement Learning

Computer Vision과 Speech Recognition에서 이뤄낸 것은 매우 큰 training sets를 활용하여 deep neural network를 효과적으로 훈련시킨 결과였다. 

대부분의 성공적인 approaches는 Stochastic Gradient Descent를 기반으로 lightweight update하여 raw input들로부터 직접 학습된 것들이었다. 

Deep Neural Network에 충분한 양의 Data를 제공하여 handcrafted된 features보다 많은 representation들을 학습할 수 있게 하였고 이를 바탕으로 RL에 대한 접근법을 생각해냈다.

목표는 RL 알고리즘을 Deep Neural Network와 연결, RGB Image에 직접적으로 작동하여 Stochastic Gradient Updates를 이용해 Traning Data를 효율적으로 처리하는 것이다.

TD-Gammon과 달리 이 논문에서는 experience replay라는 기술을 활용하였다. 

Agent가 매 step마다 했던 Episode들을 Dataset에 저장시키고, 많은 Episode들이 replay  memory에 쌓이게 되는데, 이 때 알고리즘 내부에서 샘플들이 저장된 풀로부터 임의로 하나를 샘플링하여 학습(Q-Learning, Mini-Batch)에 적용시킨다. 

이후에(experience replay 후) Agent는 e-greedy policy에 따라 행동을 수행한다.

Deep-Q Algorithm에서는 ϕ 함수를 사용하여 같은 길이의 history를 input으로 사용한다.
이러한 DQN은 기존의 Q-Learning보다 각각의 step의 Experience가 잠재적으로 더 많은 weight update에 재사용되므로, Experience를 weight update 한번만 사용하는 기존의 방법보다 훨씬 data의 측면에서 효율적이다.

연속적인 sample들로부터 학습을 진행하는 것은 데이터들 간의 상관관계 때문에 비효율적이다. 

그렇기 때문에 sample들을 e-greedy 알고리즘을 통해 randomize하여 sample들사이의 
높은 상관관계를 없애고 update의 효율성을 높인다.

기존의 on-policy를 통해 학습을 하면 매개변수가 학습된 다음 데이터 샘플을 결정한다.

이로 인해 원하지 않는 feedback loops가 발생할 수 있고, parameter들이 local minumum으로 수렴하거나 발산함을 알 수 있다.

experience replay를 사용하여 behavior distribution은 균형을 이루게 되고 parameter의 발산이나 진동을 피할 수 있어 학습을 매끄럽게 진행할 수 있다.

또한 Behavior Policy의 Parameter ϵ 과 Learning Policy의 Parameter θ 가 다르기 때문  에 experience replay를 통해 학습하기 위해서는 Q-Learning의 choice를 원활히 하는 off-policy를 사용해야 한다. 

DQN에서는 replay memory에 마지막 N개의 exprience를 저장한다.

이 후 update를 하기 위해 무작위로 Data Set으로부터 추출한다. 

이러한 접근법은 Memory Buffer가 중요한 Transition에 차별점을 두지 않으며 항상 제한된 크기 N의 버퍼에 최근의 Transition을 덮어씌운다는 부분에서 한계가 드러난다. 

또한 uniform sampling은 replay memory안의 모든 transition에 같은 중요성을 부여한다. 

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FIWiT0%2FbtqBUTcmhFb%2F8rozigtYT47Ked87sxFZsk%2Fimg.png'>
이미지 출처 - https://mangkyu.tistory.com/60

1) replay memory D와 capacity를 N으로 초기화

2) action-value function Q를 random weight로 초기화

3) episode를 M회 반복.

4) sequence s1을 t=1일때의 이미지 x1으로 초기화한 후 전처리과정을 통해 ϕ1 값을 얻는다.

5) t를 T회 반복

6) e-greedy 알고리즘을 따라 무작위 action 또는 이전에 최선의 결과를 냈던 action 중 하나를 at 선택

7) emulator에서 action at 를 수행 후 reward rt와 다음 image xt+1 를 관찰

8) 현재의 State st, 현재의 Action at, 새로운 image인 xt+1 을 st+1 로 저장한 후 st+1 에 대해 pre-processing

9) 그리고 replay memory D에 현재의 상태를 전처리한 값 ϕt, at, rt, ϕt+1 을 저장

10) D에 저장된 Sample들 중에서 minibatch의 개수만큼 random하게 선발.

11) 여기서 yj를 정의하는데, 전처리한 결과인 ϕj+1 이 목표 지점에 도달하면 rj 로, 목표지점이 아니라면 rj+γmaxa′Q(ϕj+1,a′;θ) 로 설정

12) 방정식 3을 따라 Loss Function을 정의한 후 gradient desenct 수행

## 4.1. Preprocessing and Model Architecture

### [ 1_Preprocessing ]

<img src = 'https://poqw.github.io/assets/images/dqn_7.png'>
이미지 출처 - https://poqw.github.io/DQN/

DNN에서 input에 집어넣기 전에도 전처리 과정을 거친다 (위 그림).

하지만 한 장의 이미지로는 공이 튀는 것을 표현할 수 없기 때문에 연속된 이미지를 쌓아서 Q함수에 사용 가능한 input을 만들어낸다.
<img src='https://poqw.github.io/assets/images/dqn_8.png'>
이미지 출처 - https://poqw.github.io/DQN/


공이 아래로 내려오는 것을 확인할 수 있다. 

__Preprocessing의 과정__ 을 정리하면, 
1. 주어진 history 중에서 down-sizing과 gray scale 
2. GPU환경에 맞게 square로 crop
3. last 4 frame을 stack
4. time-step마다 e-greedy방식으로 action 결정
5. 4를 토대로 reward와 $S_{(t+1)}$state를 전처리해서 experience을 구성
6. 5를 D에 저장
7. D에 저장된 sample을 mini-batch로 하여 미리 정의한 loss를 minimize하도록 GD update

### [ 2_Model Architecture ]

<img src='https://t1.daumcdn.net/cfile/tistory/99FE4F485B98F7C432'>
이미지 출처 - https://sumniya.tistory.com/18

Q-Value를 구하기 위해서

1. history와 action을 input으로 받아 output으로 Q-value
2. history를 input으로 받아 output으로 각 action에 대한 Q-Value

1 의 경우 연산의 양이 많기 때문에 2번의 방법을 사용하는데, NN의 입력으로 state를 사용하여 각 state에서 가능한 action에 대한 Q-value를 계산한다. 

이런 방식을 통해 주어진 state에서 가능한 모든 action에 대한 Q-value값을  single forward pass를 한 번만 수행하고도 처리할 수 있다.

### [ 3_DNN Architecture ]
참조 : https://adeshpande3.github.io/Deep-Learning-Research-Review-Week-2-Reinforcement-Learning

<img src='https://adeshpande3.github.io/assets/IRL11.png'>
이미지 출처 - https://adeshpande3.github.io/Deep-Learning-Research-Review-Week-2-Reinforcement-Learning

네트워크 아키텍처를 구성하는 과정을 살펴보자.

1. Neural Network의 Input은 ϕ([1]의 이미지에서 언급했음!) 를 통해 전처리된 84 * 84 * 4 이미지(4 frames)이다. 
    - 여기서 이미지는 원래 210 * 160 * 3 픽셀이다.(컬러 이미지니까)
    - 84 * 84 * 4는 전처리 된 이미지

2. 1st Hidden Layer는 input image에 stride 4를 포함하여 16 * 8 * 8로 Convolution 한 후에, rectifier non-linearity를 적용한 것이다.

3. 2nd Hidden Layer는 stride 2를 포함하여 32 * 4 *4 로 Convolution 연산을 하고 rectifier non-linearity를 적용시킨 것이다.

4. 3rd Hidden Layer는 funnly-connected되고, 256개의 rectifier 유닛으로 구성된다.

5. Output layer는 각각의 가능한 action에 대해 single output을 갖는 fully-connected linear layer이다.



---

# 5. Experiments

Experiment를 진행하면서 algorithm 및 hyper-parameter에 대한 3가지 setting이 존재한다.

1. Reward Structure: Training을 하면서 Reward Structure에 변화를 주었다. 
    - 양의 보상은 1, 음의 보상은 -1, 변화 없음은 0으로 수정함으로써 오류 도함수의 스케일을 제한하고, 모든 게임에 동일한 learning rate를 적용할 수 있었다. 
    - 하지만 이에따라  reward의 강도 또한 제한되므로 agent 성능에 제약이 생길 수 있다.

2. RMSProp Algorithm & ϵ - greedy Algorithm
    - 최적화 알고리즘으로는 크기 32의 Minibatch을 RMSProp 알고리즘에 적용한다. 
    - Behavior Policy로는 처음부터 백만번째 프레임까지는 1에서 0.1까지 동일한 비율로 감소하는 epsilone 값을 활용해 e-greedy 알고리즘을 사용하며 이후에는 0.1로 고정한다.
    
3. Frame Skipping Technique
    - Agent가 모든 Frame을 보고 Action을 취하는 것이 아니라 K번째 프레임을 보고 액션을 고르게 하고 마지막 행동은 skipped된 frames에 반복 적용시킨다.

    - 액션을 선택하기 위해선 많은 양의 계산을 필요로 하지만 한 스텝을 forward하는 것은 훨씬 적은 연산만으로 가능하다. 

    - 그러므로 이러한 기술을 적용하여 실행시간은 같지만 약 k배 더 많은 게임을 진행시킬 수 있었다. 

    - Space Invaders를 제외한 모든 게임에서 k=4 로 정해주었는데, 이 게임에서는 게임 내부의 문제로 인해 k=3으로 처리를 하였다.

## 5.1 Training and Stability


설명이 좋아 ppt 그대로 한 장 첨부합니다.
<img src= 'https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2Fd10Bl7%2FbtqBTMdLfxB%2FD7q5e2Vqvkraef24yltnfK%2Fimg.png'>
이미지 출처 - https://mangkyu.tistory.com/60

Supervised Learning에서는 training & validation sets를 통해 학습 중에 model의 성능을
쉽게 확인할 수 있었지만 Reinforcement Learning에서는 학습 중에 agent의 progress를
정확히 측정하기 어렵다. 

이 논문의 평가의 척도는 수많은 Games를 진행하여 평균화된 Reward를 토대로, Game이나 Episode에서 평균적으로 얻은 보상이기 때문에 training과정에 모인 total reward를 주기적으로 계산해야 한다.

하지만 학습을 진행하면서 Policy의 Weights를 아주 조금 변화시키는 것만으로도 state의 distribution에 큰 변화가 생기기 때문에 여기서 사용하는 평균 total reward metric은 종종 불안정하다.

Figure2에서 왼쪽의 두 그래프는  Breakout, Sequest이라는 게임에서 학습을 하는동안 
total reward의 변화를 확인할 수 있다. 

두 그래프는 보시다시피 안정적인 형상을 나타내지 않고 상당히 불규칙하다. 

하지만 Policy에 대한 Action-Value를 예측하는 Q-function은 상당히 stable한데 이 함수는 Agent가 어떤 state에서 Policy를 따랐을 때, 얼마의 discounted reward를 얻을 수 있는지를 제공한다. 

random policy를 적용하고 training하면서 fixed state를 수집하였고 이러한 state들에 대
해 maximum predicted Q의 값을 도출할 수 있다.


오른쪽의 두 그래프를 보면 Average Predicted Q가 Agent를 이용해 얻은 average total
reward보다 더 부드럽게 증가함을 볼 수 있다, 

다른 게임에서도 그래프를 그려 보았을때 유사하게 부드럽게 증가하였다. 또한 실험에서 Q 값이 발산하는 경우는 없었다. 

물론 반드시 수렴한다고 볼 순 없지만, 이러한 방식이 RL과 SGD를 사용하여 Neural        Network를 학습시킬 수 있음을 알 수 있다.

<img src = 'https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FofqNU%2FbtqBQoynkGk%2F50FaYAWTzaZENFjXV73OXk%2Fimg.png'>
이미지 출처 - https://mangkyu.tistory.com/60

## 5.2 Visualizing the Value Function

상단에서 보이는 그림인 Figure3는 Seaquest 게임에서 학습된 value function을 시각화한 것이다. 


해당 그림은 왼쪽에 적이 등장하였을 때, predicted value가 jump함을 볼 수 있다.(Point A).

적을 발견한 Agent는 적을 향해 미사일을 발사하고, 발사된 미사일이 적을 맞추려고 할 때, 
predicted value가 오름을 보여준다(Point B). 

적이 사라지게 되면 predicted value는 원래의 값으로 돌아간다(Point C). 

Figure 3는 우리의 value function이 어떻게 나아가야 할지 학습할 수 있음을 보인다.

## 5.3 Main Evaluation


Table1: 일정 단계까지 0.05의 e값을 갖는 e-greedy 알고리즘을 이용하여 학습 알고리즘을  적용한 결과를 나타낸다.

아래의 표는 HNeat와 DQN을 이용하여 Episode의 실행 결과가 가장 좋았던 경우를 나타낸다.

---

# 6. Conclusion

(논문에서 제공한 시각화가 직관적 이해에 도움이 될 것 같아 첨부합니다.)
<img src="https://adeshpande3.github.io/assets/IRL14.png">

이미지 출처 - https://adeshpande3.github.io/assets/IRL14.png

앞에서 언급했듯이 가치 함수는 기본적으로 "특정한 상황에서 얼마나 좋은지"를 측정하기 위한 것이다. 위 그림에서 (4)를 보자. 공이 움직이는 궤적과 남은 블럭의 위치를 보면 현재 매우 높은 score를 얻고 있고, 가치함수 값이 매우 높다는 것을 알 수 있다.

다양한 Atari 게임에 모두 같은 네트워크 아키텍처, 하이퍼 파라미터, 알고리즘을 사용한 것은 RL에 대한 논문에서의 접근 방식이 적절함을 뜻한다.

추가로 이 논문에서의 접근법은 7개의 게임 중 6개에서 놀라운 결과를 냈다.

## [DQN algorithm]

(추가로 DQN 알고리즘도 첨부합니다!)

<img src='https://www.intel.ai/wp-content/uploads/sites/69/deep-Q-learning-algorithm.png'>
이미지 출처 - https://www.intel.ai/demystifying-deep-reinforcement-learning/#_ftn1


---

# 8. Reference

https://adeshpande3.github.io/Deep-Learning-Research-Review-Week-2-Reinforcement-Learning (강화학습 전반에 대한 이해)

https://www.slideshare.net/ssuser75e5ed/playing-atari-with-deep-reinforcement-learning-99366224 (Atari 게임에 어떻게 적용되었는지)

https://poqw.github.io/DQN/ (DQN 정의 이해)

https://untitledtblog.tistory.com/139 (MDP 전반에 대한 이해)

https://github.com/yohan5050/MAL/wiki/%5B%EA%B0%95%ED%99%94%ED%95%99%EC%8A%B5%5D4.-Bellman-Equation-(%EB%B2%A8%EB%A7%8C-%EB%B0%A9%EC%A0%95%EC%8B%9D) (벨만 방정식 전반에 대한 이해)

https://mc.ai/introduction-reinforcement-learning-with-epsilon-greedybandit-gamealgorithm/ (입실론 그리디 알고리즘-1)

https://medium.com/analytics-vidhya/the-epsilon-greedy-algorithm-for-reinforcement-learning-5fe6f96dc870 (입실론 그리디 알고리즘-2)

# 참고한 논문 
### 1. Mastering the game of Go with deep neural networks and tree search
    D Silver 
### 2. A Deeper Look at Experience Replay
    Shangtong Zhang, Richard S. Sutton

대부분의 내용을 이해하고 싶었지만 너무 오래 걸려서.. 일부 내용만 발췌해서 본문 내용에 함께 작성했습니다.